In [7]:
import requests
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from numpy import asarray

In [5]:
custom_vision_imgurl = 'https://<resource_name>.cognitiveservices.azure.com/customvision/v3.0/Prediction/<project_id>/detect/iterations/<published_name>/image'
api_key = '<YOUR_PREDICTION_KEY>'

In [9]:
data = open('1.jpg', 'rb').read()
img = cv2.imdecode(np.array(bytearray(data), dtype='uint8'), cv2.IMREAD_COLOR)

# it's for one picture

FileNotFoundError: [Errno 2] No such file or directory: '1.jpg'

### tag-id、boundingbox

In [8]:
custom_vision_headers = {
    'Content-Type': 'application/octet-stream',
    'Prediction-Key': api_key
    }

custom_vision_resp = requests.post(url = custom_vision_imgurl,
    data = data,
    headers = custom_vision_headers).json()

NameError: name 'data' is not defined

In [ ]:
hit = pd.DataFrame(custom_vision_resp['predictions']).sort_values(by='probability',ascending=False).head(1).to_dict()

print(hit)

In [ ]:
# extract the bounding box for the detected number plate
boundingbox = list(hit['boundingBox'].values())[0]
l, t, w, h = (boundingbox['left'],
    boundingbox['top'],
    boundingbox['width'],
    boundingbox['height'])

# extract bounding box coordinates and dimensions are scaled using image dimensions
polylines1 = np.multiply([[l,t],[l+w,t],[l+w,t+h],[l,t+h]],
    [img.shape[1],img.shape[0]])

# draw polylines based on bounding box results
temp_img = cv2.polylines(img, np.int32([polylines1]),
    isClosed = True, color = (255, 255, 0), thickness = 2)

# display the original image with the plate region
plt.imshow(cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB))

### choose the license place

In [ ]:
# crop the image to the bounding box of the plate region
crop_x = polylines1[:,0].astype('uint16')
crop_y = polylines1[:,1].astype('uint16')

img_crop = img[np.min(crop_y):np.max(crop_y),
    np.min(crop_x):np.max(crop_x)]

# display the detected plate region
plt.imshow(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))

## OCR take the text
### 在調用Azure Computer Vision API之前,我們必須確保發送的圖像具有至少50像素的高度

In [ ]:
img_crop_height = img_crop.shape[0]
if img_crop_height < 50:
      pil_image = Image.fromarray(img_crop)
      img_crop_width = img_crop.shape[1]
      difference = 50 / img_crop_height
      resized_dimensions = (int(img_crop_width * difference), int(img_crop_height * difference))
      pil_image_resized = pil_image.resize(resized_dimensions)
      img_crop_resized = asarray(pil_image_resized)

      plt.imshow(cv2.cvtColor(img_crop_resized, cv2.COLOR_BGR2RGB))

### OCR tag-id、boundingbox

In [ ]:
computer_vision_imgurl =

建立這個：https://portal.azure.com/#create/Microsoft.CognitiveServicesComputerVision

建好之後用圖片轉文字，將訓練好的網址貼到上面

In [ ]:
crop_bytes = bytes(cv2.imencode('.png', img_crop_resized)[1])

# make a call to the computer_vision_imgurl
computer_vision_resp = requests.post(
    url=computer_vision_imgurl,
    data=crop_bytes,
    headers={
        'Ocp-Apim-Subscription-Key': api_key,
        'Content-Type': 'application/octet-stream'}).json()

In [ ]:
computer_vision_resp

In [ ]:
print('The plate number is {}'.format(computer_vision_resp['readResult']['content']))